In [1]:
#env main
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
import os
import h5py
from os.path import join, expanduser, basename, dirname, splitext
from glob import glob
from joblib import load, dump
from pathos.multiprocessing import ThreadPool, ProcessingPool
%load_ext autoreload
%autoreload 2
from utility import *
from config import *

In [2]:
h5name = H5NAMES[0]

## Getting borders for the low-resolution images
The steps:
1. Reading the images one by one
2. Dilating them using a kernel of size 2x2
3. getting the contours of the dilated image

NOTE: kernel size is the most important hyperparameter in this algorithm

In [ ]:
def _(args):
    i, img = args
    print(i)
    kmeans = KMeans(2)
    return kmeans.fit_predict(img.reshape(-1,1)).reshape(*img.shape)
for h5name in H5NAMES:
    data = {
        'darkfield' : np.array(h5py.File('files/%s_darkfield.h5'%h5name)['darkfield']),
        'absorption' : np.array(h5py.File('files/%s_abs.h5'%h5name)['absorption'])
    }
    f_name = join(DATA_DIR, 'area_%s.pkl'%(h5name))
    if not os.path.exists(f_name):
        low_clusters = {}
        for d_name in data:
            with ProcessingPool(4) as p:
                low_clusters[d_name] = np.array(p.map(_, enumerate(data[d_name])))
        dump(low_clusters, f_name)
    else:
        low_clusters = load(f_name)

0
16
32
48
1
17
33
2
49
18
3
34
50
4
19
35
5
20
51
6
21
36
52
7
22
37
53
8
38
23
9
54
39
24
10
25
55
11
40
12
26
41
56
13
27
14
42
57
28
15
29
58
43
64
59
30
44
65
31
60
45
80
66
61
46
67
81
62
47
68
82
63
96
69
83
112
70
97
84
98
71
113
72
99
85
114
73
100
86
115
74
101
87
116
88
75
102
76
89
117
77
103
90
118
104
78
91
119
79
92
105
128
93
120
106
94
129
121
107
95
122
130
144
108
131


In [11]:
#with ProcessingPool(4) as p:
#    low_contours = p.map(get_largest_contour, low_clusters)
#    low_contour_images = p.map(draw_largest_contour, low_clusters)
#dump(low_contours, './low_contours.pkl')
#dump(low_contour_images, './low_contour_images.pkl')

In [12]:
#files = glob('part_2__Axis_z/*jpg')
#file = random.choice(files)
#high_contours = []
#for file in files:
#    print(file)
#    img = cv2.imread(file, 0)
#    #plt.imshow(img)
#    kernel = np.ones((4,4))
#    mask = cv2.dilate((img>10).astype('uint8'), kernel, iterations=2)
#    #plt.imshow(mask*255)
#    #plt.show()
#    #_, cnts, hierarchy = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#    contours, hierarchy = cv2.findContours(mask.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#    imt = np.zeros_like(mask)
#    high_contour = max(contours, key=lambda x: len(x))
#    high_contours.append(high_contour)
#    cv2.drawContours(imt, [high_contour], -1, 1, 1)
#    #plt.imshow(imt)
#    #plt.imshow(cv2.erode((img<=10).astype('uint8'), kernel,iterations=4)*255)
#    d,f,e = dfe(file)
#    print("%s/%s"%(d, f+'_edges.png'))
#    cv2.imwrite("%s/%s"%(d, f+'_edges.png'), imt)   

In [7]:
from sklearn.cluster import KMeans

files = glob('LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0*tiff')
file = random.choice(files)
max_contours = []
def get_mask(file):
    kmeans = KMeans(3)
    print(file)
    img = cv2.imread(file, 0)
    #plt.imshow(img)
    #plt.show()
    x, y = img.shape
    clustered = kmeans.fit_predict(img.reshape(-1, 1)).reshape(img.shape)
    mask = (clustered == clustered[x//2, y//2]).astype('uint8')
    d, f, e = dfe(file)
    print("%s/%s_mask.png"%(DATA_DIR, f))
    cv2.imwrite("%s/%s_mask.png"%(DATA_DIR, f), mask)
    
def get_edges(mask):
    #plt.imshow(mask)
    contours, hierarchy = cv2.findContours(mask.astype('uint8'), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    max_contour = max(contours, key=lambda x: len(x))
    imt = np.zeros_like(mask)
    cv2.drawContours(imt, [max_contour], -1, 1, 1)
    #plt.imshow(imt)
    d,f,e = dfe(file)
    print("%s/%s"%(d, f+'_edges.png'))
    #cv2.imwrite("%s/%s"%(d, f+'_edges.png'), imt)
    return max_contour

In [8]:
from multiprocessing import Pool
with Pool(processes=8) as p:
    masks = p.map(get_mask, files)

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0162.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0509.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0925.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0744.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0545.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0471.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0439.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0453.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0471_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0241.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0453_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0970.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0509_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0958.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0439_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0760.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0744_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0265.tiff
part_analysis_result//LSU_DogboneNonH

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0559.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0947_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0715.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0921_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0561.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0187_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0627.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0363_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0429.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0096_mask.png
part_analysis_result//LSU_DogboneNonHIP_2.9um_0693_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0033.tiff
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0341.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0622_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0684.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0715_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0093.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0559_mask.png
L

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0459.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0151_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0711.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0280_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0268.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0902_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0721.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0398_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0432.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0789_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0883.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0352_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0967.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0661_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0867.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0459_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0401.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0711_mask.png
L

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0296.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0299_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0521.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0086_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0134.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0499_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0476.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0039_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0082.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0845_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0626.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0949_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0670.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0816_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0306.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0296_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0380.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0521_mask.png
L

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0613.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0954_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0081.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0165_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0465.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0664_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0490.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0810_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0527.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0879_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0448.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0968_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0805.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0204_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0348.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0613_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0085.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0081_mask.png
L

...

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0176.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0847_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0802.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0457_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0028.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0319_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0215.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0804_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0974.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0913_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0051.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0202_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0720.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0176_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0133.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0408_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0369.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0802_mask.png
L

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0871.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0308_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0254.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0286_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0430.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0473_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0324.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0713_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0173.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0835_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0952.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0065_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0153.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0994_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0310.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0254_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0109.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0871_mask.png
L

LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0087.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0621_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0422.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0102_mask.png
part_analysis_result//LSU_DogboneNonHIP_2.9um_0392_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0496.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0136_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0298.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0652_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0862.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0552_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0680.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0771_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0813.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0087_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0648.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0422_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0884.tiff
p

part_analysis_result//LSU_DogboneNonHIP_2.9um_0023_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0330.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0141_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0587.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0769_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0174.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0914_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0213.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0217_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0846.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0956_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0497.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0761_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0025.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0330_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0583.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0587_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0288.tiff
p

part_analysis_result//LSU_DogboneNonHIP_2.9um_0541_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0370.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0600_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0143.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0823_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0015.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0764_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0739.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0795_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0878.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0825_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0909.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0370_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0070.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0953_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0903.tiff
part_analysis_result//LSU_DogboneNonHIP_2.9um_0143_mask.png
LSU_NonHIP/2.9um/LSU_DogboneNonHIP_2.9um_0616.tiff
p

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [13]:
dump(max_contours, 'high_contours.pkl')

['high_contours.pkl']

In [7]:
max_contours[100].shape

(1752, 1, 2)